In [3]:
import sys, cv2
import numpy as np
#import tracktor_revised as tr
import matplotlib.pyplot as plt
import pickle
import os
import collections

from importlib import reload
import platform
import re
import datetime
import multiprocessing
from glob import glob
import logging
from tracker.tank import Tank
from tracker.tracker import Tracker
from tracker import utils

<div class="alert alert-info">
  <h2>Current Attempt</h2>
</div>

In [29]:
for x in os.listdir('.'):
    if os.path.isfile(x): 
        print('f-', x)
    elif os.path.isdir(x): 
        print('d-', x)
    elif os.path.islink(x): 
        print('l-', x)
    else: 
        print('---', x)
        
print('\n')
        
for x in os.listdir('../Fish_Tracking_Data'):
    if os.path.isfile(x): 
        print('f-', x)
    elif os.path.isdir(x): 
        print('d-', x)
    elif os.path.islink(x): 
        print('l-', x)
    else: 
        print('---', x)

d- .git
f- .gitignore
d- .ipynb_checkpoints
f- 1to1.png
f- 1tohalf.png
f- fish.pkl
f- fishGraphing.ipynb
f- fishOverlapManualCheck.ipynb
f- fishSorting.ipynb
d- frames
f- histLists.pik
f- manualOverlap.pik
d- Obsolete
f- overlapCheck.txt
f- README.md
d- report
f- sortedFish.pik
d- tracker
d- __pycache__


--- Pa_Fri_21dpf_GroupA_n2_20200619_1345.avi
--- SF_n1_t4_3012.mp4
--- SF_n2_t2_3113.mp4


In [19]:
''' Select input files to track and an output directory for each. '''

tracking_dir = 'tracker/output'

# input_files = sorted(glob('../Fish_Tracking_Data/'))
input_files = ['../Fish_Tracking_Data/SF_n2_t2_3113.mp4']

# print(len(input_files))
# display(input_files)

input_dict = {}
for input_file in input_files:
    d,fn       = os.path.split(input_file)
    fn,ext     = os.path.splitext(fn)
    output_dir = os.path.join(tracking_dir,fn)
    n_ind      = int(re.findall('_n(\d+)[^\d]*_',fn)[0])
    
    # If a repaired version exists, use that instead.
    repaired_file = os.path.join(d,'repaired',fn+'-repaired'+ext)
    input_file = repaired_file if os.path.exists(repaired_file) else input_file
    
    # If a trial file already exists, skip.
    if os.path.exists(os.path.join(output_dir,'trial.pik')):
        continue
        
    input_dict[input_file] = dict( output_dir=output_dir, n_ind=n_ind )

for f,D in input_dict.items():
    print(f)
    print('   ',D)

In [20]:
''' Create output directories. Trace tanks. '''

for input_file,D in input_dict.items():
    tracker = Tracker(input_video=input_file, **D)
    
    # Create output directory.
    tracker.init_directory()
    
    # Create a link to the input video in the output directory.
    input_shortcut = os.path.join(output_dir,'raw'+ext)
    if not 'windows' in platform.system().lower() and not os.path.exists(input_shortcut):
        os.symlink(os.path.relpath(input_file,output_dir),input_shortcut)
        
    # Trace or load tank.
    tracker.init_tank()

In [21]:
settings = dict(
    t_start          = 0,     # Time at which to start tracking, in seconds.
    t_end            = 3,    # Time at which to end tracking, in seconds.
    
    # Contour detection.
    n_pixel_blur     =  7,    # square-root of n-pixels for threshold blurring
    block_size       = 15,    # contour block size
    threshold_offset = 13,    # threshold offset for contour-finding
    min_area         = 50,    # minimum area for detection
    max_area         = 400,   # maximum area for detection
    ideal_area       = 200,   # ideal area to rank contours in first frame (default=(min_area+max_area)/2)
    max_aspect       = 20,    # maximum aspect ratio for detection
    ideal_aspect     = 3,     # ideal aspect ratio to rank contours in first frame (default=max_aspect/2)
    area_penalty     = 0.5,   # weight of area change when connecting fish across frames
    reversal_threshold = 0.5, # average frame-to-frame displacement against the director 
                            # over the last few frames to trigger a reversal. 
    live_preview     = False, # Toggle live preview of tracking.

    # Background subtraction (for naive background subtraction only).
    bkgSub_options   = dict( n_training_frames = 100, # number of frames used to compute background
                             t_start = 0, t_end = -1, # time range used to compute background
                             contrast_factor = 4 ),   # post-subtraction contrast enhancement factor
    
    # What information to draw on the tracking output video.
    video_output_options = dict( tank=True, points=False, directors=True, timestamp=True, 
                                 contours=True, contour_color=(100,255,0), contour_thickness=1 )
    )

In [22]:
def track(input_file):
    
    globals().update(settings)
    globals().update(input_dict[input_file])
    video_output_options = settings['video_output_options']
    
    utils.add_log_file(os.path.join(output_dir,'log.txt'))
    logging.info(utils.parindent+'Initializing')
    tracker = Tracker( input_video=input_file, **input_dict[input_file], **settings )
    tracker.init_all()
    tracker.save_settings()

    frames_dir = os.path.join(output_dir,'frames')

    try:
        tracker.set_frame(tracker.frame_start)
        tracker.init_live_preview()
        n_report = max(1,int((tracker.frame_end-tracker.frame_start)/50))
        for i_frame in range(tracker.frame_start, tracker.frame_end+1):
            if i_frame%n_report==0:
                percent = (i_frame-tracker.frame_start)/(tracker.frame_end-tracker.frame_start)
                logging.info(utils.parindent + f'Tracking: {tracker.get_current_timestamp()}, ' +
                             f'{tracker.get_percent_complete():4.1f}% complete')
            if tracker.get_next_frame():
                # When b evaluates to True, intermediate tracking steps are saved 
                # as images in "[output_dir]/frames".
                b = False # True # 260<tracker.frame_num<275 # 
                if b:
                    if not os.path.exists(frames_dir):
                        os.mkdir(frames_dir)
#                     cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-1_raw.png'),tracker.frame)
                tracker.subtract_background()
                tracker.mask_tank()
                tracker.detect_contours()
                tracker.connect_frames()
                if b:
#                         cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-2_masked.png'),tracker.frame)
                    video_output_options.update(directors=False)
                    tracker.draw(**video_output_options)
                    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-3_contours.png'),tracker.frame)
                    video_output_options.update(directors=True)
                tracker.draw(**video_output_options)
                if b:
                    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-4_directors.png'),tracker.frame)
                tracker.write_frame()
                if not tracker.post_frame(delay=1):
                    break
        tracker.release()
        logging.info(utils.parindent+'Saving')
        tracker.save_trial()
        logging.info(utils.parindent+'Done')

    except:
        tracker.release()
        logging.info('Failed')
        for info in sys.exc_info():
            logging.warning(info)
        %tb

    return

In [26]:
# Multi-threading doesn't work in Windows. If the OS is Windows, set n_thread to 1.
n_threads = 1 if 'windows' in platform.system().lower() else 1

if n_threads==1:

    for input_file in input_dict.keys():
        utils.reset_logging()
        utils.add_log_stream(sys.stdout)
        logging.info(input_file)
        handler = logging.root.handlers[-1]
        handler.terminator = ''
        handler.setFormatter(logging.Formatter('\r'+' '*80+'\r     %(message)s'))
        track(input_file)
    
else:
    
    utils.reset_logging()
    pool = multiprocessing.Pool(n_threads)
    pool.map(track,input_dict.keys())


globals().update(settings)
globals().update(input_dict[input_file])
video_output_options = settings['video_output_options']

utils.add_log_file(os.path.join(output_dir,'log.txt'))
logging.info(utils.parindent+'Initializing')
tracker = Tracker( input_video=input_file, **input_dict[input_file], **settings )
tracker.init_all()
tracker.save_settings()
print()

frames_dir = os.path.join(output_dir,'frames')
if not os.path.exists(frames_dir):
    os.mkdir(frames_dir)

frame_list = [ 1,2 ]
tracker.set_frame(frame_list[0])
contourList=[] #!
for i_frame in frame_list:
    tracker.get_next_frame()
    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-1_raw.png'),tracker.frame)
    tracker.subtract_background()
    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-2_subtracted.png'),tracker.frame)
    tracker.mask_tank()
    cv2.imwrite(os.path.join(frames_dir,f'{tracker.frame_num}-3_masked.png'),tracker.frame)
    tracker.detect_contours()
    tracker.connect_frames()
    
    print(tracker.frame.shape)
    print(tracker.contours)
    contourList.append(tracker.contours)
    
    break
tracker.release()
logging.info(utils.parindent+'Saving')
tracker.save_trial()
logging.info(utils.parindent+'Done')

KeyError: '../Fish_Tracking_Data/SF_n2_t2_3113.mp4'

In [27]:
print(contourList.shape)


NameError: name 'contourList' is not defined

In [ ]:
frameID_list = range(5000,n_frames,1)

#for i in frameID_list:
    
    # Load the frame.
    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
    ret,frame = cap.read()
    
    # Detect the contours.
    contours  = tr.contour_detect(frame,min_area,max_area,block_size,offset,n_pix)
    
    # Make a mask with value j inside fish number j and -1 elsewhere. 
    mask      = 0*cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) - 1
    for j in range(len(contours)):
        cv2.drawContours(mask, contours, j, j, -1)
    
    # Make and save a list of each fish's pixels.
    pixels=[]
    subFish=[]
    for j in range(len(contours)):
        pixels=(np.nonzero(mask==j))
        n =len(pixels[0]) # Number of pixels in J
        subSubFish=[]
        for k in range(n):
            row=pixels[0][k]
            col=pixels[1][k]
            rgb=frame[row,col]
            grey=round(.3*rgb[0]+.59*rgb[1]+.11*rgb[2],2)
            subSubFish.append([row,col,grey]) #Pixel
        subFish.append(np.array(subSubFish)) #Fish
    fish.append(subFish) #Frame

with open('fish.pkl','wb') as fh:
     pickle.dump(fish,fh)

<div class="alert alert-danger">
  <h2>Previous Attempt</h2>
</div>

# Creating dataset from file

## Manually creates the fish data set from the video file. This cell is currently disabled. 

In [ ]:
# List of frames to analyze.
#frameID_list = range(5000,50000,50)
frameID_list = range(5000,n_frames,1)

# Data structure to hold only the pixels belonging to a fish and their brightness.
fish = []

#Uncomment line to make this cell run, commented to force notebook to use the .pkl file for speed. (This cell takes over an hour.)
#for i in frameID_list:
    
    # Load the frame.
    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
    ret,frame = cap.read()
    
    # Detect the contours.
    contours  = tr.contour_detect(frame,min_area,max_area,block_size,offset,n_pix)
    
    # Make a mask with value j inside fish number j and -1 elsewhere. 
    mask      = 0*cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) - 1
    for j in range(len(contours)):
        cv2.drawContours(mask, contours, j, j, -1)
    
    # Make and save a list of each fish's pixels.
    pixels=[]
    subFish=[]
    for j in range(len(contours)):
        pixels=(np.nonzero(mask==j))
        n =len(pixels[0]) # Number of pixels in J
        subSubFish=[]
        for k in range(n):
            row=pixels[0][k]
            col=pixels[1][k]
            rgb=frame[row,col]
            grey=round(.3*rgb[0]+.59*rgb[1]+.11*rgb[2],2)
            subSubFish.append([row,col,grey]) #Pixel
        subFish.append(np.array(subSubFish)) #Fish
    fish.append(subFish) #Frame

with open('fish.pkl','wb') as fh:
     pickle.dump(fish,fh)

## Reads fish.pkl to create fish.

In [5]:
with open('fish.pkl','rb') as fh:
     fish=pickle.load(fh)

# Fish Overlap Check

## Checks fish for sections where the program is reading the fish as one object due to proximity, and returns a list of where the fish are not overlapping.

In [6]:
i2=0
nonOverlappingRange=[]
while i2<len(fish):
    i1=i2
    while i1<len(fish) and len(fish[i1])!=2:
        i1+=1
    i2=i1
    while i2 < len(fish) and len(fish[i2])==2:
        #find the first overlapping index
        i2+=1
    nonOverlappingRange.append([i1,i2])
print(nonOverlappingRange)

[[0, 137], [138, 204], [207, 247], [248, 371], [379, 624], [625, 916], [918, 1164], [1168, 1203], [1205, 1345], [1351, 1380], [1382, 1804], [1815, 1825], [1826, 2707], [2710, 2785], [2792, 3048], [3061, 3229], [3230, 3233], [3236, 3290], [3295, 3595], [3597, 3598], [3602, 3658], [3661, 3733], [3738, 3770], [3774, 3781], [3788, 3896], [3898, 3918], [3920, 4062], [4068, 4155], [4157, 4389], [4390, 4551], [4552, 4589], [4591, 5152], [5155, 6433], [6434, 6436], [6437, 8500], [8501, 8536], [8549, 8945], [8948, 9123], [9141, 9508], [9539, 9571], [9697, 9947], [9952, 10030], [10040, 10095], [10112, 10208], [10216, 11231], [11235, 11646], [11648, 11653], [11654, 12363], [12364, 12427], [12430, 12567], [12574, 12995], [13006, 13076], [13081, 13913], [13914, 14436], [14439, 14536], [14543, 14580], [14581, 14583], [14584, 14681], [14692, 15300], [15304, 15466], [15470, 15600], [15603, 15826], [15829, 16667], [16671, 17110], [17117, 17463], [17465, 17657], [17660, 17768], [17783, 17947], [17955, 1

## Finds the center of each fish.

In [7]:
#finding the center of each fish
#2 on fily's paper
fishMean=[]
for i in range(len(fish)):
    if len(fish[i])==1:
        fishMean.append([[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])],[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])]])
    else:
        fishMean.append([[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])],[np.mean(fish[i][1].T[0]),np.mean(fish[i][1].T[1])]])
fishMean=np.asarray(fishMean)
#fishMean[i.j.k] is the coordianate of k of the position of fjsh j in frame i

## Measures the distance between each fish during each nonoverlapping section to vertifiy that they are the same fish.

In [8]:
def swapStatus(pos,i):
    '''
    Detext swaps between consicutivve framses based on proximity.
    
    Input:
        pos:Postionts. Array with shape (Nframes,Nfish,Ndimensions),
        i: Frame index. Int.
    
    Output:
        Int. 0 if no swaps, 1 if swapped, 2 if overlapping.
    '''
    nFish=pos.shape[1] #Number of fish
    distanceMatrix=[np.linalg.norm(pos[i+1][0]-pos[i][0]),
                    np.linalg.norm(pos[i+1][1]-pos[i][1]),
                    np.linalg.norm(pos[i+1][0]-pos[i][1]),
                    np.linalg.norm(pos[i+1][1]-pos[i][0])]
    swapCriteron=(distanceMatrix[0]+distanceMatrix[1])-(distanceMatrix[2]+distanceMatrix[3])
    if abs(swapCriteron)<1e-10:
        return 2 #Overlapping
    elif swapCriteron>0:
        return 1 #Swapped
    elif swapCriteron<0:
        return 0 #Normal
    else:
        return -1

In [9]:
def normFunc(pos,i):
    '''
    Returns the distance between all possible combinations of 4 points to check whether the on or off axis diatance is larger.
    
    Input:
        i: Index, int.
    
    Output:
        1x4 list of distances.
    '''
    return [np.linalg.norm(pos[i+1][0]-pos[i][0]),
           np.linalg.norm(pos[i+1][1]-pos[i][1]),
           np.linalg.norm(pos[i+1][0]-pos[i][1]),
           np.linalg.norm(pos[i+1][1]-pos[i][0])]


fishQuadDistance=[]
for i in range(len(fishMean)-1):
    #print(i,j)
    fishQuadDistance.append(np.array(normFunc(fishMean,i)))
fishQuadDistance=np.array(fishQuadDistance)

## Unswapping the fish

In [10]:
pos=fishMean

#Checks for swqps and creates list of swaped points
i=0
I=[]
while i < len(pos):
    j=i+1
    while j<len(pos)-1 and swapStatus(pos,j)!=1:
        j+=1
    i=j+1
    while i<len(pos)-1 and swapStatus(pos,i)!=1:
        i+=1
    I.append([j+1,i+1])
    
# As a natural consequence of how the loop is structured, it creates a pair of points that are outside of the array, which we then need to delete.
del(I[-1])

In [11]:
posU=fishMean.copy()
for j,i in I:
    posU[j:i,:]=pos[j:i,[1,0]]

status=[]
for i in range(len(posU)-1):
    status.append(swapStatus(posU,i))

In [12]:
fishQuadDistance1=[]
for i in range(len(posU)-1):
    fishQuadDistance1.append(np.array(normFunc(posU,i)))
fishQuadDistance1=np.array(fishQuadDistance1)

### Unswapping the original fish file

In [13]:
fishU=fish
for i1,i2 in I:
    for i in range(i1,i2):
        if len(fishU[i])==2:
            fishU[i][0]=fish[i][1]
            fishU[i][1]=fish[i][0]

# Dumping resultant data

Dumps the sorted fishmeans and the nonOverlappingRange in a pickle file with [posU,nonOverlappingRange]

In [14]:
sortedFish=[posU,nonOverlappingRange,fishU]

with open('sortedFish.pik','wb') as f:
    pickle.dump(sortedFish,f)